In [1]:
import json
import pandas as pd
import os
import glob
from tqdm import tqdm
import re
from pandas.io.json import json_normalize
from datetime import datetime
import numpy as np

In [2]:
jason2 = pd.read_csv("jason_json_files/MILES_Garmin_trim.csv")
jason2['Start'] = pd.to_datetime(jason2['Start'], errors='coerce')
jason2

,Activity Name,Start,Distance,Time,Elevation Loss,Avg Pace,Elevation Gain,Steps,Avg Temp,Net Elevation
0,HackettstownNJ_CityHallNY_60,2016-05-23 03:07:00,70.57,16:28:07 AM,"4,058",14:00,"3,550",136836,73.4,-508
1,AndreasPA_HackettstownNJ_60,2016-05-22 06:43:00,60.76,12:44:11,"3,205",12:34,"3,123",116274,71.1,-82
2,TowerCity_Andreas_59,2016-05-21 06:47:00,50.09,11:42:08,"3,143",14:01,"3,238",99828,67.9,95
3,thompsontown_TowerCityPA_57,2016-05-20 06:48:00,52.22,12:09:25,"2,802",15:55,"3,284",103684,78.0,482
4,MtUnion_ThompsontownPA_56,2016-05-19 06:49:00,50.01,11:01:27,"2,277",13:04,"2,176",99486,76.0,-101
5,Creasson_MtUnionPA_56,2016-05-18 06:34:00,50.52,11:55:08,"4,203",14:21,"2,451",102018,68.0,-1752
6,Indiana_strongstownPA_55,2016-05-17 06:46:00,51.00,11:26:01,"3,363",13:27,"4,751",102426,69.2,1388
7,Cranberry_Indiana_PA_54,2016-05-16 06:32:00,51.07,11:53:25,"4,341",13:58,"4,252",102158,71.0,-89
8,Salineveille OH_CranberryTwp PA_53,2016-05-15 07:18:00,51.13,11:57:33,"3,081",14:02,"3,159",105956,60.6,78
9,Sugarcreek_Salineville_52,2016-05-14 06:51:00,50.40,11:41:20,2556,13:50,"2,333",99174,67.5,-223


In [3]:
jason2.dtypes

 Activity Name            object
Start             datetime64[ns]
Distance                 float64
Time                      object
Elevation Loss            object
Avg Pace                  object
Elevation Gain            object
Steps                      int64
Avg Temp                 float64
Net Elevation              int64
dtype: object

In [4]:
def express_pace_mph(row):
    minutes_part = float(row.split(":")[0])
    seconds_part = float(row.split(":")[1])
    minute_fraction = seconds_part / 60
    return 60 / (minutes_part +  minute_fraction)

jason2['mph'] = jason2['Avg Pace'].map(express_pace_mph)

jason2

,Activity Name,Start,Distance,Time,Elevation Loss,Avg Pace,Elevation Gain,Steps,Avg Temp,Net Elevation,mph
0,HackettstownNJ_CityHallNY_60,2016-05-23 03:07:00,70.57,16:28:07 AM,"4,058",14:00,"3,550",136836,73.4,-508,4.285714
1,AndreasPA_HackettstownNJ_60,2016-05-22 06:43:00,60.76,12:44:11,"3,205",12:34,"3,123",116274,71.1,-82,4.774536
2,TowerCity_Andreas_59,2016-05-21 06:47:00,50.09,11:42:08,"3,143",14:01,"3,238",99828,67.9,95,4.280618
3,thompsontown_TowerCityPA_57,2016-05-20 06:48:00,52.22,12:09:25,"2,802",15:55,"3,284",103684,78.0,482,3.769634
4,MtUnion_ThompsontownPA_56,2016-05-19 06:49:00,50.01,11:01:27,"2,277",13:04,"2,176",99486,76.0,-101,4.591837
5,Creasson_MtUnionPA_56,2016-05-18 06:34:00,50.52,11:55:08,"4,203",14:21,"2,451",102018,68.0,-1752,4.181185
6,Indiana_strongstownPA_55,2016-05-17 06:46:00,51.00,11:26:01,"3,363",13:27,"4,751",102426,69.2,1388,4.460967
7,Cranberry_Indiana_PA_54,2016-05-16 06:32:00,51.07,11:53:25,"4,341",13:58,"4,252",102158,71.0,-89,4.295943
8,Salineveille OH_CranberryTwp PA_53,2016-05-15 07:18:00,51.13,11:57:33,"3,081",14:02,"3,159",105956,60.6,78,4.275534
9,Sugarcreek_Salineville_52,2016-05-14 06:51:00,50.40,11:41:20,2556,13:50,"2,333",99174,67.5,-223,4.337349


In [5]:
jason2['Time'] = jason2['Time'].map(lambda x: (x.replace(':','.',1)))
jason2['Avg Pace'] = jason2['Avg Pace'].map(lambda x: (x.replace(':','.')))
jason2.dtypes

 Activity Name            object
Start             datetime64[ns]
Distance                 float64
Time                      object
Elevation Loss            object
Avg Pace                  object
Elevation Gain            object
Steps                      int64
Avg Temp                 float64
Net Elevation              int64
mph                      float64
dtype: object

In [6]:
jason2.to_csv("jason_garmin_details.csv")